In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy import stats
%matplotlib inline

In [2]:
housing = pd.read_csv('datasets/housing/housing.csv')

In [3]:
housing["income_cat"] = pd.cut(housing["median_income"], 
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]  
    strat_test_set = housing.loc[test_index]
    
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [4]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [5]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6 
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X, y=None):
        
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        
        return np.c_[X, rooms_per_household, population_per_household,bedrooms_per_room] 


In [6]:
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")), 
                         ('attribs_adder', CombinedAttributesAdder()), 
                         ('std_scaler', StandardScaler()), 
                        ])

cat_pipeline = Pipeline([('encoder', OneHotEncoder())
                        ])

In [7]:
housing_num = housing.drop("ocean_proximity", axis=1)
housing_cat = housing[["ocean_proximity"]] 
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs),
                                   ("cat", cat_pipeline, cat_attribs), 
                                  ]) 
 

In [8]:
housing_prepared = full_pipeline.fit_transform(housing)

# Model building

In [9]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
housing_predictions = lin_reg.predict(housing_prepared) 
lin_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
lin_rmse

68628.19819848922

In [10]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions) )
tree_rmse

0.0

In [11]:
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions) )
forest_rmse

18740.897498377428

In [50]:
svm_reg = SVR()
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions) )
svm_rmse

118580.68301157995

# cross-validation

In [13]:
scores = cross_val_score(lin_reg,
                         housing_prepared,
                         housing_labels,
                         scoring="neg_mean_squared_error",
                         cv=10) 

lin_rmse_scores = np.sqrt(-scores)
print('Mean error:',lin_rmse_scores.mean())
print('Std error:',lin_rmse_scores.std())

Mean error: 69052.46136345083
Std error: 2731.674001798347


In [12]:
scores = cross_val_score(tree_reg,
                         housing_prepared,
                         housing_labels,
                         scoring="neg_mean_squared_error",
                         cv=10) 

tree_rmse_scores = np.sqrt(-scores)
print('Mean error:',tree_rmse_scores.mean())
print('Std error:',tree_rmse_scores.std())

Mean error: 70679.96841238369
Std error: 2841.258762154916


In [15]:
scores = cross_val_score(forest_reg,
                         housing_prepared,
                         housing_labels,
                         scoring="neg_mean_squared_error",
                         cv=10) 

forest_rmse_scores = np.sqrt(-scores)
print('Mean error:',forest_rmse_scores.mean())
print('Std error:',forest_rmse_scores.std())

Mean error: 50218.782940831414
Std error: 2044.0958984374845


In [17]:
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}] 
grid_search = GridSearchCV(forest_reg, 
                           param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True) 
 
grid_search.fit(housing_prepared, housing_labels)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [25]:
mean_scores = np.sqrt(-grid_search.cv_results_["mean_test_score"])
params = grid_search.cv_results_["params"]

for mean_score, param in zip(mean_scores, params):
    print(mean_score, param)

63713.99067730326 {'max_features': 2, 'n_estimators': 3}
55473.81997336434 {'max_features': 2, 'n_estimators': 10}
52395.389596272275 {'max_features': 2, 'n_estimators': 30}
60605.22887725684 {'max_features': 4, 'n_estimators': 3}
53005.74231713559 {'max_features': 4, 'n_estimators': 10}
50645.50826104539 {'max_features': 4, 'n_estimators': 30}
59285.07707756004 {'max_features': 6, 'n_estimators': 3}
52301.5638905305 {'max_features': 6, 'n_estimators': 10}
50041.36987263269 {'max_features': 6, 'n_estimators': 30}
58710.01852837401 {'max_features': 8, 'n_estimators': 3}
51733.0031391898 {'max_features': 8, 'n_estimators': 10}
50083.513778894645 {'max_features': 8, 'n_estimators': 30}


In [42]:
ft_imps = grid_search.best_estimator_.feature_importances_ 
cat_one_hot_attribs = list(full_pipeline.named_transformers_["cat"]['encoder'].categories_[0])
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"] 
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(ft_imps, attributes), reverse=True)

[(0.32534175886641337, 'median_income'),
 (0.1480471187711867, 'INLAND'),
 (0.10588370317528001, 'pop_per_hhold'),
 (0.0798356149808007, 'bedrooms_per_room'),
 (0.07764326169397036, 'longitude'),
 (0.07058069018806314, 'latitude'),
 (0.06408073658278675, 'rooms_per_hhold'),
 (0.04415591447273799, 'housing_median_age'),
 (0.018236342688726717, 'total_rooms'),
 (0.017482068978249988, 'population'),
 (0.016601994818799412, 'total_bedrooms'),
 (0.01655484938672714, 'households'),
 (0.0080606838736648, '<1H OCEAN'),
 (0.003783659090222142, 'NEAR BAY'),
 (0.0035852861663561286, 'NEAR OCEAN'),
 (0.0001263162660146184, 'ISLAND')]

In [44]:
final_model = grid_search.best_estimator_ 
 
X_test = strat_test_set.drop("median_house_value", axis=1) 
y_test = strat_test_set["median_house_value"].copy() 
 
X_test_prepared = full_pipeline.transform(X_test) 
 
final_predictions = final_model.predict(X_test_prepared) 
 
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("Test RMSE error: ",final_rmse)

Test RMSE error:  47914.166130416765


In [51]:
confidence = 0.95 
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, 
                         loc=squared_errors.mean(), 
                         scale=stats.sem(squared_errors))) 

array([45943.92424972, 49806.53025942])